In [2]:
!pip install pymysql

     -------------------------------------- 45.0/45.0 kB 369.5 kB/s eta 0:00:00


# DATABASE CREATION WITH SQLITE3

In [4]:
# Cell 1: Install necessary libraries (run once)
!pip install pandas sqlite3 requests ipython-sql

# Cell 2: Import libraries
import sqlite3
import pandas as pd
import requests
from datetime import datetime, timedelta

# Cell 3: Connect to SQLite database
conn = sqlite3.connect('flight_database.db')
cursor = conn.cursor()
print("Database connection established.")

ERROR: Could not find a version that satisfies the requirement sqlite3 (from versions: none)
ERROR: No matching distribution found for sqlite3


Database connection established.


In [5]:
#CCreating Airports table
cursor.execute('''
CREATE TABLE IF NOT EXISTS Airports (
    airport_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    iata_code TEXT UNIQUE,
    icao_code TEXT UNIQUE,
    country TEXT,
    city TEXT,
    latitude REAL,
    longitude REAL
)
''')
conn.commit()
print("Airports table created.")

Airports table created.


In [6]:
#Insert 5 German airports
airports_data = [
    ("Berlin Brandenburg Airport", "BER", "EDDB", "Germany", "Berlin", 52.3514, 13.4937),
    ("Munich Airport", "MUC", "EDDM", "Germany", "Munich", 48.3538, 11.7861),
    ("Frankfurt Airport", "FRA", "EDDF", "Germany", "Frankfurt", 50.0333, 8.5706),
    ("Hamburg Airport", "HAM", "EDDH", "Germany", "Hamburg", 53.6304, 9.9882),
    ("Cologne Bonn Airport", "CGN", "EDDK", "Germany", "Cologne", 50.8659, 7.1427)
]

cursor.executemany('INSERT INTO Airports (name, iata_code, icao_code, country, city, latitude, longitude) VALUES (?, ?, ?, ?, ?, ?, ?)', airports_data)
conn.commit()
print("Inserted 5 German airports.")

#Creating additional tables of Airlines, Flights, FlightStatus
cursor.execute('''
CREATE TABLE IF NOT EXISTS Airlines (
    airline_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    code TEXT UNIQUE
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Flights (
    flight_id INTEGER PRIMARY KEY AUTOINCREMENT,
    flight_number TEXT NOT NULL,
    airline_id INTEGER,
    departure_airport_id INTEGER,
    arrival_airport_id INTEGER,
    scheduled_departure TEXT,
    scheduled_arrival TEXT,
    FOREIGN KEY (airline_id) REFERENCES Airlines(airline_id),
    FOREIGN KEY (departure_airport_id) REFERENCES Airports(airport_id),
    FOREIGN KEY (arrival_airport_id) REFERENCES Airports(airport_id)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS FlightStatus (
    status_id INTEGER PRIMARY KEY AUTOINCREMENT,
    flight_id INTEGER,
    actual_departure TEXT,
    actual_arrival TEXT,
    status TEXT,
    delay_minutes INTEGER,
    FOREIGN KEY (flight_id) REFERENCES Flights(flight_id)
)
''')
conn.commit()
print("Airlines, Flights, and FlightStatus tables created.")

#Insert sampling airlines and flights
cursor.execute("INSERT INTO Airlines (name, code) VALUES ('Lufthansa', 'LH')")
cursor.execute("INSERT INTO Airlines (name, code) VALUES ('Ryanair', 'FR')")

flights_data = [
    ("LH123", 1, 1, 2, "2025-03-23 08:00", "2025-03-23 09:30"),  
    ("FR456", 2, 3, 4, "2025-03-23 10:00", "2025-03-23 11:30"),  
    ("LH789", 1, 5, 1, "2025-03-23 12:00", "2025-03-23 13:45"),  #delayed
    ("FR101", 2, 2, 3, "2025-03-23 14:00", "2025-03-23 15:15"),  
    ("LH202", 1, 4, 5, "2025-03-23 16:00", "2025-03-23 17:30"),  #delayed
]

cursor.executemany('INSERT INTO Flights (flight_number, airline_id, departure_airport_id, arrival_airport_id, scheduled_departure, scheduled_arrival) VALUES (?, ?, ?, ?, ?, ?)', flights_data)

# Insert flight statuses
flight_statuses = [
    (1, "2025-03-23 08:05", "2025-03-23 09:35", "Completed", 5),
    (2, "2025-03-23 10:00", "2025-03-23 11:25", "Completed", 0),
    (3, "2025-03-23 12:10", "2025-03-23 16:00", "Completed", 135),  #Delayed > 2 hrs
    (4, "2025-03-23 14:05", "2025-03-23 15:20", "Completed", 5),
    (5, "2025-03-23 16:15", "2025-03-23 19:00", "Completed", 150),  #Delayed > 2 hrs
]

cursor.executemany('INSERT INTO FlightStatus (flight_id, actual_departure, actual_arrival, status, delay_minutes) VALUES (?, ?, ?, ?, ?)', flight_statuses)
conn.commit()
print("Inserted sample flights and statuses.")

Inserted 5 German airports.
Airlines, Flights, and FlightStatus tables created.
Inserted sample flights and statuses.


In [7]:
query1 = '''
SELECT f.flight_number, a1.name AS departure, a2.name AS arrival, f.scheduled_departure, f.scheduled_arrival
FROM Flights f
JOIN Airports a1 ON f.departure_airport_id = a1.airport_id
JOIN Airports a2 ON f.arrival_airport_id = a2.airport_id
WHERE a1.iata_code = 'BER'
'''
flights_from_ber = pd.read_sql_query(query1, conn)
print("Flights from BER:")
print(flights_from_ber)

Flights from BER:
  flight_number                   departure         arrival  \
0         LH123  Berlin Brandenburg Airport  Munich Airport   

  scheduled_departure scheduled_arrival  
0    2025-03-23 08:00  2025-03-23 09:30  


In [8]:
query2 = '''
SELECT f.flight_number, a1.name AS departure, a2.name AS arrival, fs.delay_minutes
FROM Flights f
JOIN FlightStatus fs ON f.flight_id = fs.flight_id
JOIN Airports a1 ON f.departure_airport_id = a1.airport_id
JOIN Airports a2 ON f.arrival_airport_id = a2.airport_id
WHERE fs.delay_minutes > 120
'''
delayed_flights = pd.read_sql_query(query2, conn)
print("Flights delayed by more than 2 hours:")
print(delayed_flights)

Flights delayed by more than 2 hours:
  flight_number             departure                     arrival  \
0         LH789  Cologne Bonn Airport  Berlin Brandenburg Airport   
1         LH202       Hamburg Airport        Cologne Bonn Airport   

   delay_minutes  
0            135  
1            150  


In [9]:
flight_number = 'LH789'
query3 = '''
SELECT f.flight_number, al.name AS airline, a1.name AS departure, a2.name AS arrival, fs.actual_departure, fs.actual_arrival, fs.delay_minutes
FROM Flights f
JOIN Airlines al ON f.airline_id = al.airline_id
JOIN Airports a1 ON f.departure_airport_id = a1.airport_id
JOIN Airports a2 ON f.arrival_airport_id = a2.airport_id
JOIN FlightStatus fs ON f.flight_id = fs.flight_id
WHERE f.flight_number = ?
'''
flight_details = pd.read_sql_query(query3, conn, params=(flight_number,))
print(f"Details for flight {flight_number}:")
print(flight_details)

Details for flight LH789:
  flight_number    airline             departure                     arrival  \
0         LH789  Lufthansa  Cologne Bonn Airport  Berlin Brandenburg Airport   

   actual_departure    actual_arrival  delay_minutes  
0  2025-03-23 12:10  2025-03-23 16:00            135  


# API ENDPOINT CREATION WITH AVIATIONSTACK


In [4]:
import requests
import sqlite3
from datetime import datetime
import logging

#Setup logging for api
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

# AviationStack API key
API_KEY = "f3c053ef669497c6ad9f887ca41e4da5"

#Connecting database
def get_db_connection():
    conn = sqlite3.connect('flight_database.db')
    conn.row_factory = sqlite3.Row
    return conn

#AviationStack fetching data
def fetch_realtime_flight_data():
    url = f"http://api.aviationstack.com/v1/flights?access_key={API_KEY}&limit=10"  # Limit to 10 for demo
    try:
        response = requests.get(url, timeout=10)
        if response.status_code == 200:
            logger.info("Successfully fetched data from AviationStack.")
            return response.json()['data']  # Return list of flights
        else:
            logger.error(f"API error: {response.status_code} - {response.text}")
            return None
    except requests.RequestException as e:
        logger.error(f"Error fetching data: {e}")
        return None

#store or update flight data in the database
def store_flight_data(flights):
    conn = get_db_connection()
    cursor = conn.cursor()

    for flight in flights:
        flight_number = flight.get('flight', {}).get('number', 'Unknown')
        departure_iata = flight.get('departure', {}).get('iata', '')
        arrival_iata = flight.get('arrival', {}).get('iata', '')
        scheduled_departure = flight.get('departure', {}).get('scheduled', '')
        scheduled_arrival = flight.get('arrival', {}).get('scheduled', '')
        actual_arrival = flight.get('arrival', {}).get('actual', None)
        status = flight.get('flight_status', 'Unknown')

        #Calculating delay
        delay_minutes = None
        if actual_arrival and scheduled_arrival:
            try:
                sched = datetime.strptime(scheduled_arrival, "%Y-%m-%dT%H:%M:%S%z")
                actual = datetime.strptime(actual_arrival, "%Y-%m-%dT%H:%M:%S%z")
                delay = (actual - sched).total_seconds() / 60
                delay_minutes = int(delay) if delay > 0 else 0
            except ValueError as e:
                logger.warning(f"Date parsing error for flight {flight_number}: {e}")

        #Get airport IDs
        cursor.execute("SELECT airport_id FROM Airports WHERE iata_code = ?", (departure_iata,))
        dep_id = cursor.fetchone()[0] if cursor.fetchone() else None
        cursor.execute("SELECT airport_id FROM Airports WHERE iata_code = ?", (arrival_iata,))
        arr_id = cursor.fetchone()[0] if cursor.fetchone() else None

        if not dep_id or not arr_id:
            logger.warning(f"Airport not found: {departure_iata} or {arrival_iata}")
            continue

        #Check if flight exists
        cursor.execute("SELECT flight_id FROM Flights WHERE flight_number = ?", (flight_number,))
        flight_id = cursor.fetchone()

        if flight_id:
            flight_id = flight_id[0]
            #Update existing flight
            cursor.execute("""
                UPDATE Flights SET scheduled_departure = ?, scheduled_arrival = ?
                WHERE flight_id = ?""", (scheduled_departure, scheduled_arrival, flight_id))
            cursor.execute("""
                UPDATE FlightStatus SET actual_arrival = ?, status = ?, delay_minutes = ?
                WHERE flight_id = ?""", (actual_arrival, status, delay_minutes, flight_id))
            logger.info(f"Updated flight {flight_number}.")
        else:
            #Insert new flight (assuming airline_id=1, else 0)
            cursor.execute("""
                INSERT INTO Flights (flight_number, airline_id, departure_airport_id, arrival_airport_id, 
                                    scheduled_departure, scheduled_arrival)
                VALUES (?, 1, ?, ?, ?, ?)""", 
                (flight_number, dep_id, arr_id, scheduled_departure, scheduled_arrival))
            flight_id = cursor.lastrowid
            cursor.execute("""
                INSERT INTO FlightStatus (flight_id, actual_arrival, status, delay_minutes)
                VALUES (?, ?, ?, ?)""", 
                (flight_id, actual_arrival, status, delay_minutes))
            logger.info(f"Inserted new flight {flight_number}.")

    conn.commit()
    conn.close()

#execution
if __name__ == "__main__":
    flights = fetch_realtime_flight_data()
    if flights:
        store_flight_data(flights)
    else:
        logger.error("No flight data retrieved.")

2025-03-24 00:34:43,070 - DEBUG - Starting new HTTP connection (1): api.aviationstack.com:80
2025-03-24 00:34:45,963 - DEBUG - http://api.aviationstack.com:80 "GET /v1/flights?access_key=f3c053ef669497c6ad9f887ca41e4da5&limit=10 HTTP/1.1" 200 None
2025-03-24 00:34:45,969 - INFO - Successfully fetched data from AviationStack.
2025-03-24 00:34:45,974 - WARNING - Airport not found: KHG or QSZ
2025-03-24 00:34:45,976 - WARNING - Airport not found: PKX or ENH
2025-03-24 00:34:45,978 - WARNING - Airport not found: CAI or IST
2025-03-24 00:34:45,981 - WARNING - Airport not found: PVG or CGQ
2025-03-24 00:34:45,983 - WARNING - Airport not found: TNA or WUH
2025-03-24 00:34:45,986 - WARNING - Airport not found: HKG or SIN
2025-03-24 00:34:45,988 - WARNING - Airport not found: CNS or MEL
2025-03-24 00:34:45,990 - WARNING - Airport not found: KHH or KIX
2025-03-24 00:34:45,992 - WARNING - Airport not found: ROK or BNE
2025-03-24 00:34:45,995 - WARNING - Airport not found: WNZ or KWE


NOTE: AIRPORT NOT FOUND COULD BE BECAUSE THE SAMPLE DATA THAT WAS GENERATED, THE ABBREVEATIONS MIGHT NOT BE THE SAME